In [21]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd

In [22]:
pages = range(0, 250, 25)
list(pages)

[0, 25, 50, 75, 100, 125, 150, 175, 200, 225]

In [23]:
def download_all_htmls():
    htmls = []
    for i in pages:
        url = f'https://movie.douban.com/top250?start={i}&filter='
        print('craw html:', url)
        response = requests.get(url,
                        headers={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)'})
        if response.status_code != 200:
            return None
        htmls.append(response.text)
    return htmls

In [24]:
htmls = download_all_htmls()

craw html: https://movie.douban.com/top250?start=0&filter=
craw html: https://movie.douban.com/top250?start=25&filter=
craw html: https://movie.douban.com/top250?start=50&filter=
craw html: https://movie.douban.com/top250?start=75&filter=
craw html: https://movie.douban.com/top250?start=100&filter=
craw html: https://movie.douban.com/top250?start=125&filter=
craw html: https://movie.douban.com/top250?start=150&filter=
craw html: https://movie.douban.com/top250?start=175&filter=
craw html: https://movie.douban.com/top250?start=200&filter=
craw html: https://movie.douban.com/top250?start=225&filter=


In [92]:
def parse_single_html(html):

    soup = bs(html, 'html.parser')
    lists = (
        soup.find('div', class_='article')
            .find('ol', class_='grid_view')
            .find_all('div', class_='item')
    )
    datas = []
    for each in lists:
        index = each.find('div', class_='pic').find('em').string
        info = each.find('div', class_='info')
        name = info.find('div', class_='hd').find('span', class_='title').string
        bd = info.find('div', class_='bd')
        stars = (
              bd.find('div', class_='star')
                .find_all('span')
        )
        rating_num = stars[1].get_text()
        comments = stars[3].get_text()
        DAT = bd.find('p').get_text()
        
        datas.append({
            '电影排名':index,
            '电影名称':name,
            '导演/主演/时间':DAT,
            '评分':rating_num,
            '评价人数':comments.replace("人评价", ""),
        })
    return datas

In [95]:
import pprint
pprint.pprint(parse_single_html(htmls[9]))

[{'导演/主演/时间': '\n'
              '                            导演: 丹尼斯·甘塞尔 Dennis '
              'Gansel\xa0\xa0\xa0主演: 尤尔根·沃格尔 Jürgen Vogel ...\n'
              '                            2008\xa0/\xa0德国\xa0/\xa0剧情 惊悚\n'
              '                        ',
  '电影名称': '浪潮',
  '电影排名': '226',
  '评价人数': '195836',
  '评分': '8.7'},
 {'导演/主演/时间': '\n'
              '                            导演: 史蒂芬·戴德利 Stephen '
              'Daldry\xa0\xa0\xa0主演: 凯特·温丝莱特 Kate Winslet ...\n'
              '                            2008\xa0/\xa0美国 德国\xa0/\xa0剧情 爱情\n'
              '                        ',
  '电影名称': '朗读者',
  '电影排名': '227',
  '评价人数': '360858',
  '评分': '8.6'},
 {'导演/主演/时间': '\n'
              '                            导演: 詹姆斯·沃德·布柯特 James Ward '
              'Byrkit\xa0\xa0\xa0主演: 艾米丽·芭尔多尼 Em...\n'
              '                            2013\xa0/\xa0美国 英国\xa0/\xa0科幻 悬疑 '
              '惊悚\n'
              '                        ',
  '电影名称': '彗星来的那一夜',
  '电影排名': '228',
 

In [94]:
all_datas = []
for html in htmls:
    all_datas.extend(parse_single_html(html))

In [85]:
all_datas

[{'电影排名': '1',
  '电影名称': '肖申克的救赎',
  '导演/主演/时间': '\n                            导演: 弗兰克·德拉邦特 Frank Darabont\xa0\xa0\xa0主演: 蒂姆·罗宾斯 Tim Robbins /...\n                            1994\xa0/\xa0美国\xa0/\xa0犯罪 剧情\n                        ',
  '评分': '9.7',
  '评价人数': '1928541'},
 {'电影排名': '2',
  '电影名称': '霸王别姬',
  '导演/主演/时间': '\n                            导演: 陈凯歌 Kaige Chen\xa0\xa0\xa0主演: 张国荣 Leslie Cheung / 张丰毅 Fengyi Zha...\n                            1993\xa0/\xa0中国大陆 中国香港\xa0/\xa0剧情 爱情 同性\n                        ',
  '评分': '9.6',
  '评价人数': '1418848'},
 {'电影排名': '3',
  '电影名称': '阿甘正传',
  '导演/主演/时间': '\n                            导演: 罗伯特·泽米吉斯 Robert Zemeckis\xa0\xa0\xa0主演: 汤姆·汉克斯 Tom Hanks / ...\n                            1994\xa0/\xa0美国\xa0/\xa0剧情 爱情\n                        ',
  '评分': '9.5',
  '评价人数': '1465104'},
 {'电影排名': '4',
  '电影名称': '这个杀手不太冷',
  '导演/主演/时间': '\n                            导演: 吕克·贝松 Luc Besson\xa0\xa0\xa0主演: 让·雷诺 Jean Reno / 娜塔莉·波特曼 ...\n                            1

In [78]:
df = pd.DataFrame(all_datas)

In [79]:
df

,电影排名,电影名称,导演/主演/时间,评分,评价人数
0,1,肖申克的救赎,\n 导演: 弗兰克·德拉邦特 Fra...,9.7,1928541
1,2,霸王别姬,\n 导演: 陈凯歌 Kaige Ch...,9.6,1418848
2,3,阿甘正传,\n 导演: 罗伯特·泽米吉斯 Rob...,9.5,1465104
3,4,这个杀手不太冷,\n 导演: 吕克·贝松 Luc Be...,9.4,1660405
4,5,美丽人生,\n 导演: 罗伯托·贝尼尼 Robe...,9.5,927708
...,...,...,...,...,...
245,246,四个春天,\n 导演: 陆庆屹 Lu Qing ...,8.9,110976
246,247,E.T. 外星人,\n 导演: 史蒂文·斯皮尔伯格 St...,8.6,247290
247,248,撞车,\n 导演: 保罗·哈吉斯 Paul ...,8.6,243303
248,249,变脸,\n 导演: 吴宇森 John Woo...,8.5,308614


In [81]:
df.to_excel('DouBanMovie Top 250.xls')